In [5]:
!pip install torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
from glob import glob
import librosa
import librosa.display
import numpy as np
import pandas as pd
import pickle
import torch
import torchaudio
import IPython.display as ipd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [43]:
max_pad_len = 1000

def extract_feature(file_name):
    print('file name :', file_name)
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0,0), (0, pad_width)), mode='constant')
        print(mfccs.shape)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        print(e);
        return None
    
#     return padded_mfccs
    return mfccs

In [33]:
audio_path = '/content/drive/MyDrive/전주해커톤/사용할_데이터/사운드/sound/괘종시계 작동/괘종시계/괘종시계 작동_괘종시계_Sm0000002.wav'
y, sr = librosa.load(audio_path)

In [34]:
print(y, sr)
print((len(y)))
print(len(y)/sr)

ipd.Audio(y, rate=sr)

[ 4.5846984e-09 -2.5163935e-08 -1.5512642e-08 ...  2.4624076e-06
  1.1318167e-06  0.0000000e+00] 22050
282593
12.816009070294784


In [35]:
file_name = audio_path

In [36]:
audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

In [37]:
mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
print(mfccs)

[[-5.9010559e+02 -5.9010559e+02 -5.9010559e+02 ... -4.6587097e+02
  -5.1000092e+02 -5.7180432e+02]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  1.3843475e+01
   2.3688393e+01  1.3373207e+01]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -4.8521824e+01
  -4.0783939e+01 -4.8764071e+00]
 ...
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -5.9319763e+00
   9.9688768e-05 -9.3036175e-01]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -4.1275182e+00
  -1.3438517e+00  1.0087783e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  4.0082617e+00
   3.4268398e+00 -1.5423918e-01]]


In [41]:
max_pad_len = 1000
pad_width = max_pad_len - mfccs.shape[1]
pad_width

448

In [42]:
mfccs = np.pad(mfccs, pad_width=((0,0), (0, pad_width)), mode='constant')
mfccs.shape

(40, 1000)

In [40]:
print(audio, sample_rate)

[ 3.7575463e-09 -2.4262540e-08 -1.6582778e-08 ...  2.5005734e-06
  1.1037641e-06  0.0000000e+00] 22050


In [19]:
y.shape

(231367,)

In [44]:
extract_feature(audio_path)

file name : /content/drive/MyDrive/전주해커톤/사용할_데이터/사운드/sound/괘종시계 작동/괘종시계/괘종시계 작동_괘종시계_Sm0000002.wav
(40, 1000)


array([[-590.1056, -590.1056, -590.1056, ...,    0.    ,    0.    ,
           0.    ],
       [   0.    ,    0.    ,    0.    , ...,    0.    ,    0.    ,
           0.    ],
       [   0.    ,    0.    ,    0.    , ...,    0.    ,    0.    ,
           0.    ],
       ...,
       [   0.    ,    0.    ,    0.    , ...,    0.    ,    0.    ,
           0.    ],
       [   0.    ,    0.    ,    0.    , ...,    0.    ,    0.    ,
           0.    ],
       [   0.    ,    0.    ,    0.    , ...,    0.    ,    0.    ,
           0.    ]], dtype=float32)

In [ ]:
fulldatasetpath = '/content/drive/MyDrive/전주해커톤/사용할_데이터/사운드/sound/작두질/작두'
metadata = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')
features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    file_name = os.path.join(os.path.abspath(fulldatasetpath),
				'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
		class_label = row["classID"]
    data = extract_feature(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])